# Tracing 101

Step through this notebook to understand how tracing works in Generative AI Toolkit.

The Generative AI Toolkit comes with these tracers out-of-the-box:


In [1]:
from generative_ai_toolkit.tracer import (
    NoopTracer,
    HumanReadableTracer,
    InMemoryTracer,
    StructuredLogsTracer,
    TeeTracer,
)
from generative_ai_toolkit.tracer.otlp import OtlpTracer
from generative_ai_toolkit.tracer.dynamodb import DynamoDbTracer

import time
import random

### `InMemoryTracer`

Use the in-memory tracer for testing and development:


In [2]:
in_memory_tracer = InMemoryTracer(
    memory_size=1000  # Store max 1000 traces, before discarding older ones
)

# Context, added to all traces:
in_memory_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with in_memory_tracer.trace("parent") as parent_span:
    parent_span.add_attribute("foo", "bar")
    parent_span.add_attribute(
        "inherited.foo",
        "bar",
        inheritable=True,  # Inheritable attributes propagate to child spans
    )
    time.sleep(0.1)

    # Nested spans become child spans, that point to the parent (parent_span_id):
    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("bar", "foo")
        time.sleep(0.1)

for trace in in_memory_tracer.get_traces():
    print(trace)
    print()

Trace(span_name='parent', span_kind='INTERNAL', trace_id='3bad5a7ffd1d9cdd1911c8c997d64eab', span_id='dedc749975a8f981', parent_span_id=None, started_at=datetime.datetime(2025, 4, 15, 11, 9, 54, 843436, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 11, 9, 55, 46329, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL', trace_id='3bad5a7ffd1d9cdd1911c8c997d64eab', span_id='49d704b41ed7ec4c', parent_span_id='dedc749975a8f981', started_at=datetime.datetime(2025, 4, 15, 11, 9, 54, 943975, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 11, 9, 55, 46285, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)



### Printing a human-readable version of traces during development

In the following example we add attributes that Generative AI Toolkit understands. It will use these to present traces in a way that is nicer to the human eye:


In [3]:
conversation_id = random.randint(0, 1000000)

with in_memory_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in in_memory_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id  # filter traces by conversation id
    }
):
    print(trace.as_human_readable())
    print()

[60ca579122f6f8dd7e96e448b1c01450/root/18953791421dc7e8] MyAgent SERVER 2025-04-15T11:09:55.052Z - parent (ai.conversation.id='765390' ai.auth.context='user123')


[60ca579122f6f8dd7e96e448b1c01450/18953791421dc7e8/d609dddcfac33d7e] MyAgent INTERNAL 2025-04-15T11:09:55.155Z - child (ai.trace.type='tool-invocation' ai.conversation.id='765390' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `HumanReadableTracer`

You can also use the `HumanReadableTracer` that will log traces in human readable form to stdout, which is useful during development.

Note that traces are logged when the span ends, so parent spans are logged after child spans (this is true for all tracers):


In [4]:
import sys

human_readable_tracer = HumanReadableTracer(stream=sys.stdout)

human_readable_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with human_readable_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with human_readable_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

[9e0ddf09c6dee26a40dabea6cd18f2bd/06642bd8bf7840a5/b9a170790d35ed99] MyAgent INTERNAL 2025-04-15T11:09:55.376Z - child (ai.trace.type='tool-invocation' ai.conversation.id='765390' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!

[9e0ddf09c6dee26a40dabea6cd18f2bd/root/06642bd8bf7840a5] MyAgent SERVER 2025-04-15T11:09:55.271Z - parent (ai.conversation.id='765390' ai.auth.context='user123')



### `StructuredLogsTracer`

Use the `StructuredLogsTracer` to log traces to stdout as JSON:


In [5]:
structured_logs_tracer = StructuredLogsTracer(stream=sys.stdout)

structured_logs_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with structured_logs_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with structured_logs_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"child","span_kind":"INTERNAL","trace_id":"aa083321db19146ad77bf6c8a475996b","span_id":"433160e7f8beb002","parent_span_id":"2e6853bbd4ff1250","started_at":"2025-04-15 11:09:55.595650+00:00","ended_at":"2025-04-15 11:09:55.696896+00:00","attributes":{"ai.trace.type":"tool-invocation","ai.tool.input":"Hello, world!","ai.tool.output":"World, hello!","ai.conversation.id":765390,"ai.auth.context":"user123"},"span_status":"UNSET","resource_attributes":{"service.name":"MyAgent"},"scope":{"name":"generative-ai-toolkit","version":"current"}}}
{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"parent","span_kind":"SERVER","trace_id":"aa083321db19146ad77bf6c8a475996b","span_id":"2e6853bbd4ff1250","parent_span_id":null,"started_at":"2025-04-15 11:09:55.495131+00:00","ended_at":"2025-04-15 11:09:55.697449+00:00","attributes":{"ai.conversation.id":765390,"ai.auth.context":"user123"},"span_status

### `DynamoDbTracer`

Use the `DynamoDbTracer` to store traces to DynamoDB.

To use this tracer, you should have created a table with partition key `pk` (string) and sort key `sk` (string).

If you want to support getting traces by conversation ID, the table must have a GSI with partition key `conversation_id` (string) and sort key `sk` (string).

For example, here's how to create such a table:


In [6]:
!aws dynamodb create-table \
  --table-name MyTracesTable \
  --attribute-definitions \
    AttributeName=pk,AttributeType=S \
    AttributeName=sk,AttributeType=S \
    AttributeName=conversation_id,AttributeType=S \
  --key-schema \
    AttributeName=pk,KeyType=HASH \
    AttributeName=sk,KeyType=RANGE \
  --billing-mode PAY_PER_REQUEST \
  --global-secondary-indexes '[{"IndexName":"conversation_index","KeySchema":[{"AttributeName":"conversation_id","KeyType":"HASH"},{"AttributeName":"sk","KeyType":"RANGE"}],"Projection":{"ProjectionType":"ALL"}}]'


An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: MyTracesTable


Then, use that table in the `DynamoDbTracer`:


In [7]:
conversation_id = random.randint(0, 1000000)
auth_context = "user123"

ddb_tracer = DynamoDbTracer(
    table_name="MyTracesTable",
    identifier="MyAgent",
    conversation_id_gsi_name="conversation_index",
)

ddb_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with ddb_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with ddb_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in ddb_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace.as_human_readable())
    print()

[147a1289ee2bab31579025727a030815/root/1d63b4ddb25c0c61] MyAgent SERVER 2025-04-15T11:09:57.151Z - parent (ai.conversation.id='956992' ai.auth.context='user123')


[147a1289ee2bab31579025727a030815/1d63b4ddb25c0c61/841c247d3cf208a4] MyAgent INTERNAL 2025-04-15T11:09:57.256Z - child (ai.trace.type='tool-invocation' ai.conversation.id='956992' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `OtlpTracer`

The `OtlpTracer` logs traces in Open Telemetry protobuf format. It expects you to run an Open Telemetry collector, that it can send the traces to. By default, it expects the collector to be run on localhost port 4318.

You can use the `OtlpTracer` to send traces to AWS X-Ray. To make that work, you can run the [ADOT collector](https://github.com/aws-observability/aws-otel-collector) locally:


In [8]:
# Create the ADOT config file:

yaml_content = """\
receivers:
  otlp:
    protocols:
      http:
        endpoint: 0.0.0.0:4318

processors:
  batch/traces:
    timeout: 10s
    send_batch_size: 50

exporters:
  awsxray:
    region: eu-central-1
    indexed_attributes:
      - ai.conversation.id

service:
  pipelines:
    traces:
      receivers: [otlp]
      processors: [batch/traces]
      exporters: [awsxray]
"""

with open("adot-config.yaml", "w") as f:
    f.write(yaml_content)

Run the ADOT collector in the background. Note that the following example assumes `AWS_REGION`, `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_SESSION_TOKEN` are available as environment variables:


In [9]:
!docker run --rm -d --name adot-collector \
  -p 4318:4318 \
  -e AWS_REGION \
  -e AWS_ACCESS_KEY_ID \
  -e AWS_SECRET_ACCESS_KEY \
  -e AWS_SESSION_TOKEN \
  -v $(pwd)/adot-config.yaml:/etc/collector-config.yaml \
  public.ecr.aws/aws-observability/aws-otel-collector:latest \
  --config=/etc/collector-config.yaml
!sleep 2 # wait for the collector to start
!docker logs adot-collector

f42f76932426c3458619c77a7137fa86d7c7e1adee6dae9eaaec07dc3c32b559
2025/04/15 11:09:59 ADOT Collector version: v0.43.1
2025/04/15 11:09:59 found no extra config, skip it, err: open /opt/aws/aws-otel-collector/etc/extracfg.txt: no such file or directory
2025/04/15 11:09:59 attn: users of the `datadog`, `logzio`, `sapm`, `signalfx` exporter components. please refer to https://github.com/aws-observability/aws-otel-collector/issues/2734 in regards to an upcoming ADOT Collector breaking change
2025-04-15T11:09:59.873Z	info	service@v0.117.0/service.go:164	Setting up own telemetry...
2025-04-15T11:09:59.873Z	info	telemetry/metrics.go:70	Serving metrics	{"address": "localhost:8888", "metrics level": "Normal"}
2025-04-15T11:09:59.874Z	info	service@v0.117.0/service.go:230	Starting aws-otel-collector...	{"Version": "v0.43.1", "NumCPU": 2}
2025-04-15T11:09:59.874Z	info	extensions/extensions.go:39	Starting extensions...
2025-04-15T11:09:59.874Z	info	otlpreceiver@v0.117.0/otlp.go:169	Starting HTTP ser

Then, send traces to AWS X-Ray by using the `OtlpTracer`:


In [10]:
otlp_tracer = OtlpTracer()

otlp_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with otlp_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", "123456", inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with otlp_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

If that seems to work, but you don't see traces appear in AWS X-Ray, check the ADOT container logs. E.g. there may be a permission issue:


In [ ]:
!docker logs adot-collector

### `NoopTracer`

Use the no-operation tracer when you don't want traces:


In [12]:
noop_tracer = NoopTracer()
with noop_tracer.trace("noop") as span:
    span.add_attribute("foo", "bar")

# nothing was logged

### `TeeTracer`

Use the `TeeTracer` to send traces to multiple tracers at once.

Note that the first tracer you add, will be the one that `get_traces()` will be delegated to. So if you want to use that method, use a tracer that supports it.

Add tracers like this:


In [13]:
tee_tracer = TeeTracer()

# E.g. the DynamoDBTracer supports get_traces(), so add that first:
tee_tracer.add_tracer(ddb_tracer)

tee_tracer.add_tracer(human_readable_tracer)

# This is of course useless, but added for the sake of the example:
tee_tracer.add_tracer(noop_tracer)

Then, use the `TeeTracer` as any other tracer:


In [14]:
conversation_id = random.randint(0, 1000000)
auth_context = "user456"

tee_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with tee_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with tee_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


print("==== from DynamoDB: ====")
for trace in tee_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace)
    print()

[2396646821570baab0588b1e8c7a2682/e80248d102421237/380a960fef0195f3] MyAgent INTERNAL 2025-04-15T11:10:01.163Z - child (ai.trace.type='tool-invocation' ai.conversation.id='269160' ai.auth.context='user456')
       Input: Hello, world!
      Output: World, hello!

[2396646821570baab0588b1e8c7a2682/root/e80248d102421237] MyAgent SERVER 2025-04-15T11:10:01.058Z - parent (ai.conversation.id='269160' ai.auth.context='user456')

==== from DynamoDB: ====
Trace(span_name='parent', span_kind='SERVER', trace_id='2396646821570baab0588b1e8c7a2682', span_id='e80248d102421237', parent_span_id=None, started_at=datetime.datetime(2025, 4, 15, 11, 10, 1, 58239, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 11, 10, 1, 305444, tzinfo=datetime.timezone.utc), attributes={'ai.conversation.id': 269160, 'ai.auth.context': 'user456'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL',